In [2]:
# init
import pandas as pd
import numpy as np
from tqdm import tqdm
# import nltk
# nltk.download()

Import Data

In [ ]:
import glob
import os

pubmed = []

# import pubmed
docList = glob.glob(os.path.join(os.getcwd(), "Datasets/pubmed/", "*.txt"))

for docPath in docList:
    # get doc file name
    docName = os.path.basename(docPath).split('.')[0]
    
    with open(docPath) as doc:
        # insert [class, docs, feature]
        pubmed.append([docName[:3], docName, doc.read().replace('\n', ' ')])

# print(pubmed)

# make dataframe
dataframe = pd.DataFrame(data=pubmed, columns=['class', 'document', 'feature']) 

# export pubmed raw
dataframe.to_csv('pubmed/raw.csv', index=False)

Dataframe raw

In [ ]:
# read csv
dataRaw = pd.read_csv('pubmed/raw.csv')
# get feature
features = dataRaw.loc[:, 'feature']
dataRaw

Preprocesing

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# cleaning
def cleaning(features):
    result = []
    for feature in features:
        regex = re.sub(r'[^a-zA-Z\s]', '', feature)
        result.append(regex)
    return result

# case folding
def caseFolding(features):
    result = []
    for feature in features:
        lower = feature.lower()
        result.append(lower)
    return result

# tokenization
def tokenization(features):
    result = []
    for feature in features:
        token = word_tokenize(feature)
        result.append(token)
    return result

# stopwords removal
def stopWords(features):
    result = []
    stopWords = set(stopwords.words('english'))
    for token in features:
        cleanedFeature = [feature for feature in token if feature not in stopWords]
        result.append(cleanedFeature)
    return result

In [ ]:
# cleaning
clean = cleaning(features)
# print(clean)

case = caseFolding(clean)
# print(casefolding)

token = tokenization(case)
# print(tokenization)

preprocessed = stopWords(token)
# print(preprocessedFeature)

# export pubmed clean
for i in range(len(preprocessed)):
    dataRaw.loc[i, 'feature'] = ' '.join(preprocessed[i])
dataRaw.to_csv('pubmed/clean.csv', index=False)

In [ ]:
# read csv
dataClean = pd.read_csv('pubmed/clean.csv')
# get feature
features = dataClean.loc[:, 'feature']
dataClean

Feature Forming

In [ ]:
from textblob import TextBlob

# BOAW
dataClean.rename(columns={'feature': 'BOAW'}, inplace=True)

for i in tqdm(range(len(features))):
    # BON
    dataClean.loc[i,'BON'] = ' '.join(TextBlob(features[i]).noun_phrases)
    # BONA
    dataClean.loc[i,'BONA'] = ' '.join([word for (word, tag) in TextBlob(features[i]).tags if tag[:2]=='NN' or tag[:2]=='JJ'])

# print(dataClean)

dataClean.to_csv('pubmed/formed.csv', index=False)

In [3]:
# read csv
dataFormed = pd.read_csv('pubmed/formed.csv')
# get features
classes = dataFormed.loc[:, 'class']
boaw = dataFormed.loc[:, 'BOAW']
bon = dataFormed.loc[:, 'BON']
bona = dataFormed.loc[:, 'BONA']
dataFormed

,class,document,BOAW,BON,BONA
0,ALZ,ALZ1,reduced amounts immunoreactive somatostatin te...,amounts immunoreactive somatostatin temporal c...,reduced amounts immunoreactive somatostatin te...
1,ALZ,ALZ10,diagnostic criteria primary neuronal degenerat...,diagnostic criteria primary neuronal degenerat...,diagnostic criteria primary neuronal degenerat...
2,ALZ,ALZ100,cacetylcholine synthesis ccarbon dioxide produ...,cacetylcholine synthesis ccarbon dioxide produ...,cacetylcholine synthesis ccarbon dioxide produ...
3,ALZ,ALZ1000,pattern reading deterioration dementia alzheim...,pattern reading deterioration dementia alzheim...,pattern deterioration dementia alzheimer type ...
4,ALZ,ALZ101,cerebral blood flow metabolic rate oxygen gluc...,cerebral blood flow metabolic rate oxygen gluc...,cerebral blood flow metabolic rate oxygen gluc...
...,...,...,...,...,...
3995,HIV,HIV3996,major histocompatibility complex genes influen...,major histocompatibility complex genes outcome...,major histocompatibility complex genes influen...
3996,HIV,HIV3997,hiv infection cohort haemophilic patients cour...,hiv infection cohort haemophilic patients cour...,hiv infection cohort haemophilic patients cour...
3997,HIV,HIV3998,evolution definition aids main classifications...,evolution definition aids main classifications...,evolution definition aids main classifications...
3998,HIV,HIV3999,human immunodeficiency virus glycoproteins gp ...,human immunodeficiency virus glycoproteins gp ...,human immunodeficiency virus glycoproteins gp ...


Term Weighting

In [23]:
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# TF
def tf(features):
    # get tf weights
    tfVec = CountVectorizer()
    result = tfVec.fit_transform(features)

    # define weights into dataframe
    featureName = tfVec.get_feature_names_out()
    featureWeight = result.todense().tolist()
    df = pd.DataFrame(featureWeight, columns=featureName)

    # counting weights into dictionary
    tf = dict.fromkeys(featureName, 0)
    for word in featureName:
      arr = np.array(df.loc[:, word])
      tf[word] = np.sum(arr)
    return tf

# tfx = tf(bona)
# print(tfx)

# TF-IDF
def tf_idf(features):
    # get tf-idf weights
    tfIdfVec = TfidfVectorizer()
    result = tfIdfVec.fit_transform(features)

    # define weights into dataframe
    featureName = tfIdfVec.get_feature_names_out()
    featureWeight = result.todense().tolist()
    df = pd.DataFrame(featureWeight, columns=featureName)
    
    # counting weights into dictionary
    tfIdf = dict.fromkeys(featureName, 0)
    for word in featureName:
      arr = np.array(df.loc[:, word])
      tf[word] = np.sum(arr)
    return tfIdf

# idfx = tf_idf(bona)
# print(idfx)

# TF-IDF-ICF
def icf(word):
  C = []
  ct = []

  for i in classes:
    # count class
    if i not in C:
      C.append(i)

  # for row in 
    # count class term
    if i not in ct:
       if 
  

  
  for word, val in idfDict.items():
      icf = math.log(len(C) / float(val))
  
  return icf

def tf_idf_icf(features):
    tfIdf = tf_idf(features)
    tfIdfIcf = dict.fromkeys(list(tfIdf.keys()), 0)

    for word, val in tfIdf:
        tfIdfIcf[word] = val * icf(word)
        # break
    return tfIdfIcf

# icfx = tf_idf_icf(features)
# print(icfx)

TypeError: 'function' object does not support item assignment

Feature Selection

Clustering

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline